In [142]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import datetime as dt
import difflib

In [130]:
my_data = pd.read_csv('data/training-dump.csv', sep=',', dtype={
    "banned_by": str,
    "no_follow": bool,
    "link_id": str,
    "gilded": bool,
    "author": str,
    "author_verified": bool,
    "author_comment_karma": np.float64,
    "author_link_karma": np.float64,
    "num_comments": np.float64,
    "created_utc": np.float64,
    "score": np.float64,
    "over_18": bool,
    "body": str,
    "downs": np.float64,
    "is_submitter": bool,
    "num_reports": np.float64,
    "controversiality": np.float64,
    "quarantine": str,
    "ups": np.float64,
    "is_bot": bool,
    "is_troll": bool,
    "recent_comments": str})

In [131]:
# delete columns that have missing data or won't have meaningful values in real-time data
columns = ['banned_by', 'downs', 'quarantine', 'num_reports', 'num_comments', 'score', 'ups', 'controversiality', 'gilded']
my_data.drop(columns, inplace=True, axis=1)

# drop duplicates
my_data.drop_duplicates(inplace=True)

# format columns
my_data['created_utc'] = pd.to_datetime(my_data['created_utc'].values, unit='s')
my_data['body'] = my_data['body'].str.slice(stop=200).fillna('')

# add our new stats columns
my_data['recent_num_comments'] = pd.Series(np.zeros(len(my_data.index), np.int64))
my_data['recent_num_last_30_days'] = pd.Series(np.zeros(len(my_data.index), np.int64))
my_data['recent_avg_no_follow'] = pd.Series(np.zeros(len(my_data.index), np.float64))
my_data['recent_avg_gilded'] = pd.Series(np.zeros(len(my_data.index), np.float64))
my_data['recent_avg_responses'] = pd.Series(np.zeros(len(my_data.index), np.float64))
my_data['recent_percent_neg_score'] = pd.Series(np.zeros(len(my_data.index), np.float64))
my_data['recent_avg_score'] = pd.Series(np.zeros(len(my_data.index), np.float64))
my_data['recent_min_score'] = pd.Series(np.zeros(len(my_data.index), np.float64))
my_data['recent_avg_controversiality'] = pd.Series(np.zeros(len(my_data.index), np.float64))
my_data['recent_avg_ups'] = pd.Series(np.zeros(len(my_data.index), np.float64))
my_data['recent_avg_diff_ratio'] = pd.Series(np.zeros(len(my_data.index), np.float64))
my_data['recent_max_diff_ratio'] = pd.Series(np.zeros(len(my_data.index), np.float64))
my_data['recent_avg_sentiment_polarity'] = pd.Series(np.zeros(len(my_data.index), np.float64))
my_data['recent_min_sentiment_polarity'] = pd.Series(np.zeros(len(my_data.index), np.float64))

In [132]:
def diff_ratio(_a, _b):
    return difflib.SequenceMatcher(a=_a,b=_b).ratio()

def last_30(a, b):
    a_datetime = pd.to_datetime(a, unit='s')  # Convert 'a' to a datetime object
    return a_datetime - dt.timedelta(days=30) < pd.to_datetime(b, unit='s')

num = 0;

def calc_stats(comment):
    # track progress
    global num 
    num += 1
    if(num % 1000 == 0): print(num)
        
    recent_comments = pd.read_json(comment['recent_comments'], dtype={
        "banned_by": str,
        "no_follow": bool,
        "link_id": str,
        "gilded": np.float64,
        "author": str,
        "author_verified": bool,
        "author_comment_karma": np.float64,
        "author_link_karma": np.float64,
        "num_comments": np.float64,
        "created_utc": np.float64,
        "score": np.float64,
        "over_18": bool,
        "body": str,
        "downs": np.float64,
        "is_submitter": bool,
        "num_reports": np.float64,
        "controversiality": np.float64,
        "quarantine": bool,
        "ups": np.float64})
    comment['recent_num_comments'] = len(recent_comments)
    
    if(len(recent_comments) > 0):
        comment['recent_num_last_30_days'] = recent_comments['created_utc'].apply(lambda x: last_30(comment['created_utc'], x)).sum()
        comment['recent_avg_no_follow'] = recent_comments['no_follow'].mean()
        comment['recent_avg_gilded'] = recent_comments['gilded'].mean()
        comment['recent_avg_responses'] = recent_comments['num_comments'].mean()
        comment['recent_percent_neg_score'] = recent_comments['score'].apply(lambda x: x < 0).mean() * 100
        comment['recent_avg_score'] = recent_comments['score'].mean()
        comment['recent_min_score'] = recent_comments['score'].min()
        comment['recent_avg_controversiality'] = recent_comments['controversiality'].mean()
        comment['recent_avg_ups'] = recent_comments['ups'].mean()
        diff = recent_comments['body'].str.slice(stop=200).fillna('').apply(lambda x: diff_ratio(comment['body'], x))
        comment['recent_avg_diff_ratio'] = diff.mean()
        comment['recent_max_diff_ratio'] = diff.max()
        scores = pd.concat([recent_comments['body'], pd.Series(comment['body'])]).apply(lambda x: TextBlob(x).sentiment.polarity)
        comment['recent_avg_sentiment_polarity'] = scores.mean()
        comment['recent_min_sentiment_polarity'] = scores.min()
        
    return comment

new_data = my_data.apply(calc_stats, axis=1)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [133]:
new_data

,no_follow,link_id,author,author_verified,author_comment_karma,author_link_karma,created_utc,over_18,body,is_submitter,...,recent_avg_responses,recent_percent_neg_score,recent_avg_score,recent_min_score,recent_avg_controversiality,recent_avg_ups,recent_avg_diff_ratio,recent_max_diff_ratio,recent_avg_sentiment_polarity,recent_min_sentiment_polarity
0,True,t3_2l5szg,ADHDbot,False,-6.0,1.0,2014-11-03 15:02:38,False,"As per the rules in the side bar, yes or no qu...",False,...,0.00,0.0,0.00,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.00
1,True,t3_2l61gs,ADHDbot,False,-6.0,1.0,2014-11-03 16:21:27,False,Meme and image posts are not allowed on this s...,False,...,1.00,0.0,1.00,1.0,0.00,1.00,0.025000,0.025000,0.133333,0.10
2,True,t3_2l7ma8,ADHDbot,False,-6.0,1.0,2014-11-04 00:21:05,False,"As per the rules in the side bar, yes or no qu...",False,...,1.00,0.0,1.00,1.0,0.00,1.00,0.517500,1.000000,0.144444,0.10
3,True,t3_2l7t5h,ADHDbot,False,-6.0,1.0,2014-11-04 01:23:18,False,"As per the rules in the side bar, yes or no qu...",False,...,1.00,0.0,1.00,1.0,0.00,1.00,0.678333,1.000000,0.150000,0.10
4,True,t3_2l900k,ADHDbot,False,-6.0,1.0,2014-11-04 10:15:33,False,We cannot and will not diagnose anyone. You n...,False,...,1.00,0.0,1.00,1.0,0.00,1.00,0.035000,0.050000,0.130000,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267027,True,t3_52doh1,haiku_robot,True,159092.0,1.0,2016-09-12 15:41:06,False,"holy shit, this news hit Killary much ...",False,...,721.30,0.0,6.30,0.0,0.05,6.30,0.386731,0.472973,0.129154,-0.70
267028,False,t3_52drz9,haiku_robot,True,159092.0,1.0,2016-09-12 15:41:11,False,There's a small dick joke somewhere in...,False,...,684.15,0.0,6.25,0.0,0.05,6.25,0.343547,0.430233,0.117250,-0.70
267029,True,t3_52d5po,haiku_robot,True,159092.0,1.0,2016-09-12 15:41:14,False,This guy is going to the share the sam...,False,...,645.55,0.0,6.80,0.0,0.05,6.80,0.366457,0.455090,0.106932,-0.70
267030,False,t3_52drtm,haiku_robot,True,159092.0,1.0,2016-09-12 15:41:17,False,Why does this have so many upvotes? Se...,False,...,667.65,0.0,6.90,0.0,0.05,6.90,0.400011,0.506329,0.130146,-0.70


In [138]:
# delete columns that have missing data or won't have meaningful values in real-time data
columns = ['link_id', 'author', 'created_utc', 'body', 'recent_comments', 'is_troll', 'link_id', 'author',  'body', 'recent_comments', 'created_utc']
new_data.drop(columns, inplace=True, axis=1)

# drop duplicates
new_data.drop_duplicates(inplace=True)

new_data.dtypes

no_follow                           bool
author_verified                     bool
author_comment_karma             float64
author_link_karma                float64
over_18                             bool
is_submitter                        bool
is_bot                              bool
recent_num_comments                int64
recent_num_last_30_days          float64
recent_avg_no_follow             float64
recent_avg_gilded                float64
recent_avg_responses             float64
recent_percent_neg_score         float64
recent_avg_score                 float64
recent_min_score                 float64
recent_avg_controversiality      float64
recent_avg_ups                   float64
recent_avg_diff_ratio            float64
recent_max_diff_ratio            float64
recent_avg_sentiment_polarity    float64
recent_min_sentiment_polarity    float64
dtype: object

In [148]:
X = new_data.drop(['is_bot'], axis=1)
y = new_data['is_bot'].values
display(X.head())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
model = tree.DecisionTreeClassifier(max_depth=3, class_weight={False:1, True:2.5})
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

y_true = y_test

matrix = pd.crosstab(y_true, y_pred, rownames=['True'],
                     colnames=['Predicted'], margins=True)
print(matrix)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Mcc:", metrics.matthews_corrcoef(y_test, y_pred))
print("F1 :", metrics.f1_score(y_test, y_pred, average=None))
print("Recall :", metrics.recall_score(y_test, y_pred, average=None))
print("Precision:", metrics.precision_score(y_test, y_pred, average=None))

feature_imp = pd.Series(
    model.feature_importances_,
    index=X.columns).sort_values(ascending=False)

print(feature_imp)

no_nans = new_data.dropna()
X = no_nans.drop(['is_bot'], axis=1)
y = no_nans['is_bot'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
model2 = RandomForestClassifier(max_depth = 3, class_weight = {False:1, True:2.5})
model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)

y_true = y_test

matrix = pd.crosstab(y_true, y_pred, rownames=['True'],
                     colnames=['Predicted'], margins=True)
print(matrix)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Mcc:", metrics.matthews_corrcoef(y_test, y_pred))
print("F1 :", metrics.f1_score(y_test, y_pred, average=None))
print("Recall :", metrics.recall_score(y_test, y_pred, average=None))
print("Precision:", metrics.precision_score(y_test, y_pred, average=None))

feature_imp = pd.Series(
    model2.feature_importances_,
    index=X.columns).sort_values(ascending=False)

print(feature_imp)

,no_follow,author_verified,author_comment_karma,author_link_karma,over_18,is_submitter,recent_num_comments,recent_num_last_30_days,recent_avg_no_follow,recent_avg_gilded,recent_avg_responses,recent_percent_neg_score,recent_avg_score,recent_min_score,recent_avg_controversiality,recent_avg_ups,recent_avg_diff_ratio,recent_max_diff_ratio,recent_avg_sentiment_polarity,recent_min_sentiment_polarity
0,True,False,-6.0,1.0,False,False,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.000000,0.00
1,True,False,-6.0,1.0,False,False,1,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.025000,0.025,0.133333,0.10
2,True,False,-6.0,1.0,False,False,2,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.517500,1.000,0.144444,0.10
3,True,False,-6.0,1.0,False,False,3,3.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.678333,1.000,0.150000,0.10
4,True,False,-6.0,1.0,False,False,4,4.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.035000,0.050,0.130000,0.05


Predicted  False   True    All
True                          
False       1301    667   1968
True           4  62855  62859
All         1305  63522  64827
Accuracy: 0.9896493744890247
Mcc: 0.807494065260284
F1 : [0.79498931 0.99469066]
Recall : [0.66107724 0.99993637]
Precision: [0.99693487 0.9894997 ]
author_comment_karma             0.678007
recent_avg_diff_ratio            0.166560
author_verified                  0.154238
recent_max_diff_ratio            0.001196
no_follow                        0.000000
recent_percent_neg_score         0.000000
recent_avg_sentiment_polarity    0.000000
recent_avg_ups                   0.000000
recent_avg_controversiality      0.000000
recent_min_score                 0.000000
recent_avg_score                 0.000000
recent_avg_responses             0.000000
recent_avg_gilded                0.000000
recent_avg_no_follow             0.000000
recent_num_last_30_days          0.000000
recent_num_comments              0.000000
is_submitter            